In [ ]:
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
import seaborn as sns

import logging

import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize

import gc

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import zipfile
import os

# Path to your zip file in Google Drive
zip_path = '/content/drive/My Drive/Projects/NLP P1/blogs.zip'

# Destination folder to extract files
extract_path = '/content/drive/My Drive/Projects//NLP P1/'

# Unzip
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Unzipped successfully!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Unzipped successfully!


In [ ]:
# Reading in the data

df = pd.read_csv('/content/drive/My Drive/Projects/NLP P1/blogtext.csv')

# Analysing Data

print(df.head())
print('\n')
print('-'*60)
print('\n')
df.info()


        id gender  age              topic      sign          date  \
0  2059027   male   15            Student       Leo   14,May,2004   
1  2059027   male   15            Student       Leo   13,May,2004   
2  2059027   male   15            Student       Leo   12,May,2004   
3  2059027   male   15            Student       Leo   12,May,2004   
4  3581210   male   33  InvestmentBanking  Aquarius  11,June,2004   

                                                text  
0             Info has been found (+/- 100 pages,...  
1             These are the team members:   Drewe...  
2             In het kader van kernfusie op aarde...  
3                   testing!!!  testing!!!            
4               Thanks to Yahoo!'s Toolbar I can ...  


------------------------------------------------------------


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 681284 entries, 0 to 681283
Data columns (total 7 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   i

In [ ]:
df.shape

(681284, 7)

In [ ]:
list(df.columns)
#df.columns

['id', 'gender', 'age', 'topic', 'sign', 'date', 'text']

In [ ]:
print(df.topic.value_counts())
print('\n')
print('-'*60)
print('\n')
print("Total number of unique topics : ", df.topic.unique().size)

topic
indUnk                     251015
Student                    153903
Technology                  42055
Arts                        32449
Education                   29633
Communications-Media        20140
Internet                    16006
Non-Profit                  14700
Engineering                 11653
Law                          9040
Publishing                   7753
Science                      7269
Government                   6907
Consulting                   5862
Religion                     5235
Fashion                      4851
Marketing                    4769
Advertising                  4676
BusinessServices             4500
Banking                      4049
Chemicals                    3928
Telecommunications           3891
Accounting                   3832
Military                     3128
Museums-Libraries            3096
Sports-Recreation            3038
HumanResources               3010
RealEstate                   2870
Transportation               2326
Manufact

In [ ]:

# Checking for null values

df.isna().sum()

,0
id,0
gender,0
age,0
topic,0
sign,0
date,0
text,0


In [ ]:
# Drop columns with no significance here

df = df.drop(['id','gender','age','sign','date'], axis = 1)

In [ ]:
df.describe

<bound method NDFrame.describe of                     topic                                               text
0                 Student             Info has been found (+/- 100 pages,...
1                 Student             These are the team members:   Drewe...
2                 Student             In het kader van kernfusie op aarde...
3                 Student                   testing!!!  testing!!!          
4       InvestmentBanking               Thanks to Yahoo!'s Toolbar I can ...
...                   ...                                                ...
681279            Student         Dear Susan,  I could write some really ...
681280            Student         Dear Susan,  'I have the second yeast i...
681281            Student         Dear Susan,  Your 'boyfriend' is fuckin...
681282            Student         Dear Susan:    Just to clarify, I am as...
681283            Student         Hey everybody...and Susan,  You might a...

[681284 rows x 2 columns]>

In [ ]:
# Changing 'topic' column data type

catagorical_coloumn = ["topic"]

df[catagorical_coloumn] = df[catagorical_coloumn].astype("category")

In [ ]:
df.head()

,topic,text
0,Student,"Info has been found (+/- 100 pages,..."
1,Student,These are the team members: Drewe...
2,Student,In het kader van kernfusie op aarde...
3,Student,testing!!! testing!!!
4,InvestmentBanking,Thanks to Yahoo!'s Toolbar I can ...


**Outcome Of Data Analysis**

1.No null values in the Data set

2. Total number of unique topics : 40

3. There are 6 different columns - 'id', 'gender', 'age', 'topic', 'sign', 'date', 'text'

4. Columns to be dropped as they dont hold significance in our current classification - id, gender, age, sign, date

In [ ]:
!pip install langdetect

In [ ]:
# Using langdetect to detect language for each review

from langdetect import detect

def detect_english(text):
  try:
    return detect(text) == 'en'
  except:
    return False


In [ ]:
df_english = []
for i in df.index:
  df_english.append(detect_english(df['text'].iloc[i]))

In [ ]:
# Count of values with df_english array

pd.Series(df_english).value_counts()

,count
True,651468
False,29816


In [ ]:
df['df_english'] = pd.Series(df_english)
df.head()

,topic,text,df_english
0,Student,"Info has been found (+/- 100 pages,...",True
1,Student,These are the team members: Drewe...,False
2,Student,In het kader van kernfusie op aarde...,True
3,Student,testing!!! testing!!!,True
4,InvestmentBanking,Thanks to Yahoo!'s Toolbar I can ...,True


In [ ]:
# Filtering the reviews detected as english by langdetect

df_english = pd.DataFrame()
df_english = df.loc[df['df_english'] == True]

df_english = df_english.drop(['df_english'], axis = 1)
df_english.shape

(651486, 2)

In [ ]:
df_english.dtypes

,0
topic,category
text,object


In [ ]:
log = logging.getLogger(__name__)
log.addHandler(logging.NullHandler())

# Ensure df_english is a DataFrame before proceeding with text processing
df_english = df.loc[df['df_english'] == True].drop(['df_english'], axis = 1)

# Text data pre-processing and cleanup

def remove_special_characters(series):
    """Remove special characters from a pandas Series of text"""
    log.info("Removing special characters from series")
    return series.str.replace(r'[^A-Za-z0-9 ]', '', regex=True)

def remove_numbers(series):
    """Remove numbers from a pandas Series of text"""
    log.info("Removing numbers from series")
    return series.str.replace(r'\d+', ' ', regex=True)

# Ensure the 'text' column is filled with empty strings for any NaN values
# and then convert to string type to handle any potential non-string objects safely.
# This prevents errors from operations on non-string types (like floats for NaN)
# and ensures the column contains only string objects or empty strings.
df_english['text'] = df_english['text'].fillna('').astype(str)

df_english['text']= remove_special_characters(df_english['text'])
df_english['text'] = remove_numbers(df_english['text'])

In [ ]:
df_english.memory_usage().sum()

np.int64(11076348)

In [ ]:
df_english_1 = df_english.sample(frac =.10)
df_english_1.head(100)

,topic,text
143118,Student,LastMinute Interview Tips Iv...
348474,Military,I think someone needs to slap me ...
634900,Student,I think Im going to stop tell...
98257,Arts,I am bipolar This whole thing could be...
335214,Arts,plush toys are all the rage for bir...
...,...,...
438763,Technology,Then there are times when I t...
539866,Communications-Media,Not only a staple of WCW programming a ...
282173,Non-Profit,Youre so soft Josh Abby can ...
355469,Student,Be sure of this O young ambition all mo...


In [ ]:
df_english_1.memory_usage().sum()

np.int64(1108891)

In [ ]:
df_english_1.shape


(65147, 2)

In [ ]:
df_english_1.isna().sum()

,0
topic,0
text,0


In [ ]:

# Text pre-processing

def lowercase(df):
  #df.astype('str')
  logging.info("Converting dataframe to lowercase")
  lowercase_df = df.apply(lambda x: x.lower())
  return lowercase_df

In [ ]:
df_english_1['text'] = lowercase(df_english_1['text'])
df_english_1.head()

,topic,text
143118,Student,lastminute interview tips iv...
348474,Military,i think someone needs to slap me ...
634900,Student,i think im going to stop tell...
98257,Arts,i am bipolar this whole thing could be...
335214,Arts,plush toys are all the rage for bir...


In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# Remove stop words from reviews

stops = set(stopwords.words('english'))

df_english_1['text'] = df_english_1['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stops)]))
df_english_1.head()

,topic,text
143118,Student,lastminute interview tips ive picking office c...
348474,Military,think someone needs slap hard mustve watched m...
634900,Student,think im going stop telling potential employer...
98257,Arts,bipolar whole thing could cycle
335214,Arts,plush toys rage birthday presents got ein cowb...


In [ ]:

def remove_whitespace(df):
  log.info("Removing extra whitespace from dataframe")
  # replace more than 1 space with 1 space
  merged_spaces = pd.Series(df).str.replace(r"\s\s+",' ')
  # delete beginning and trailing spaces
  trimmed_spaces = pd.DataFrame(merged_spaces).apply(lambda x: x.strip() if isinstance(x, str) else x)
  return trimmed_spaces


In [ ]:
df_english_1['text'] = remove_whitespace(df_english_1['text'])

In [ ]:
type(df_english_1['text'])

pandas.core.series.Series

In [ ]:
type(df_english_1['text'])

pandas.core.series.Series

In [ ]:
gc.collect()
df_english_1.head()

,topic,text
143118,Student,lastminute interview tips ive picking office c...
348474,Military,think someone needs slap hard mustve watched m...
634900,Student,think im going stop telling potential employer...
98257,Arts,bipolar whole thing could cycle
335214,Arts,plush toys rage birthday presents got ein cowb...


In [ ]:

df_english_1.to_csv('df_english_file.csv', index = False, encoding = 'utf-8-sig', chunksize=10000)

In [ ]:
df_english_1 = pd.read_csv('/content/df_english_file.csv')

In [ ]:
# Checking for any new null values

df_english_1.isna().sum()

,0
topic,0
text,16


In [ ]:
# Dropping The Null Values

df_english_1 = df_english_1.dropna()
df_english_1.isna().sum()

,0
topic,0
text,0


In [ ]:
# Independent & Target variable creation

from sklearn.model_selection import train_test_split

X = df_english_1.text
y = df_english_1.topic

In [ ]:
# Target variable values

y.value_counts()

,count
topic,
indUnk,24121
Student,14708
Technology,4088
Arts,3135
Education,2800
Communications-Media,1925
Internet,1594
Non-Profit,1341
Engineering,1090


In [ ]:
# Import label encoder
from sklearn import preprocessing

# label_encoder object knows how to understand word labels.
label_encoder = preprocessing.LabelEncoder()

# Encode labels in categorical columns
y_encoded = label_encoder.fit_transform(y)

In [ ]:
type(X), X.shape, type(y_encoded), y_encoded.shape

(pandas.core.series.Series, (65131,), numpy.ndarray, (65131,))

In [ ]:

X_train, X_test, y_train, y_test =train_test_split(X, y_encoded, random_state=42,
                                                   test_size = 0.1,
                                                  shuffle = True)

In [ ]:
type(X_train), type(y_train)


(pandas.core.series.Series, numpy.ndarray)

In [ ]:

y_train

array([39, 25, 39, ..., 39, 29, 35])

In [ ]:
print("shape of training set :", X_train.shape)
print("shape of test set :", X_test.shape)

shape of training set : (58617,)
shape of test set : (6514,)


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

X_train_copy = X_train
y_train_copy = y_train

X_test_copy = X_test
y_test_copy = y_test

stops = list(stopwords.words('english')) # Convert the set to a list

vectorizer = TfidfVectorizer(stop_words = stops)
vectorizer.fit(X_train_copy)

X_train_copy = vectorizer.transform(X_train_copy)
X_test_copy = vectorizer.transform(X_test_copy)
X_train_copy, X_test_copy

(<Compressed Sparse Row sparse matrix of dtype 'float64'
 	with 4741965 stored elements and shape (58617, 248809)>,
 <Compressed Sparse Row sparse matrix of dtype 'float64'
 	with 518437 stored elements and shape (6514, 248809)>)

In [ ]:
X_train_copy

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 4741965 stored elements and shape (58617, 248809)>

In [ ]:
X_train_copy.shape, type(X_train_copy)

((58617, 248809), scipy.sparse._csr.csr_matrix)

In [ ]:
from sklearn.metrics import classification_report

def performance_metrics(y_test,y_predict):
  print(classification_report(y_test,y_predict))

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

model_mnb = MultinomialNB()

model_mnb.fit(X_train_copy, y_train_copy)
y_predict_copy = model_mnb.predict(X_test_copy)

print('Multinomial Naive Bayes Model Accuracy : ',accuracy_score(y_test_copy, y_predict_copy))
print('\n')

performance_metrics(y_test_copy, y_predict_copy)

Multinomial Naive Bayes Model Accuracy :  0.3713540067546822


              precision    recall  f1-score   support

           0       0.00      0.00      0.00        44
           1       0.00      0.00      0.00        43
           2       0.00      0.00      0.00        13
           3       0.00      0.00      0.00        11
           4       0.00      0.00      0.00       308
           5       0.00      0.00      0.00        11
           6       0.00      0.00      0.00        41
           7       0.00      0.00      0.00        16
           8       0.00      0.00      0.00        40
           9       0.00      0.00      0.00        31
          10       0.00      0.00      0.00       199
          11       0.00      0.00      0.00         8
          12       0.00      0.00      0.00        51
          13       0.00      0.00      0.00       277
          14       0.00      0.00      0.00       107
          15       0.00      0.00      0.00         7
          16      

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
!pip install gensim

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 41.6 MB/s eta 0:00:00


In [ ]:
import gensim
from gensim.models import Word2Vec

X_train_arr = X_train.to_numpy()
X_test_arr = X_test.to_numpy()

In [ ]:
type(X_train_arr), X_train_arr.shape

(numpy.ndarray, (58617,))

In [ ]:
import gensim
from gensim.models import Word2Vec

X_train_arr = X_train.to_numpy()
X_test_arr = X_test.to_numpy()

# Tokenize X_train_arr for Word2Vec input
tokenized_X_train_arr = [text.split() for text in X_train_arr]
Word2Vec_model_Xtrained_skipgram = Word2Vec(tokenized_X_train_arr, vector_size = 150, workers = 4, sg = 1)
X_train_wv_skipgram = X_train.apply(lambda text: np.mean([Word2Vec_model_Xtrained_skipgram.wv[word] for word in text.split() if word in Word2Vec_model_Xtrained_skipgram.wv]))

/usr/local/lib/python3.12/dist-packages/numpy/_core/fromnumeric.py:3596: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.12/dist-packages/numpy/_core/_methods.py:138: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [ ]:
type(X_train_wv_skipgram), X_train_wv_skipgram.shape

(pandas.core.series.Series, (58617,))

In [ ]:
type(y_train), y_train.shape

(numpy.ndarray, (58617,))

In [ ]:
np.unique(y_train).sum()

np.int64(780)

In [ ]:
y_train = pd.Series(y_train)
#type(y_train)
train_skipgram_data = pd.concat([X_train_wv_skipgram.reset_index(drop=True), y_train.reset_index(drop=True)], axis=1)
train_skipgram_data.head()

,text,0
0,-0.000317,39
1,NaN,25
2,NaN,39
3,NaN,13
4,NaN,17


In [ ]:
train_skipgram_data.shape
print('-'*60)
print('Null Values Post Skip Gram Vectorization in train data:\n', train_skipgram_data.isna().sum())
train_skipgram_data = train_skipgram_data.dropna()
print('-'*60)
print('Shape of Vectorized Train Set post removing Null Values :',train_skipgram_data.shape)

------------------------------------------------------------
Null Values Post Skip Gram Vectorization in train data:
 text    49135
0           0
dtype: int64
------------------------------------------------------------
Shape of Vectorized Train Set post removing Null Values : (9482, 2)


In [ ]:
train_skipgram_data = train_skipgram_data.drop_duplicates()
train_skipgram_data.shape

(2961, 2)

In [ ]:
import gensim
from gensim.models import Word2Vec

# Tokenize X_test_arr for Word2Vec input
tokenized_X_test_arr = [text.split() for text in X_test_arr]
Word2Vec_model_Xtest_skipgram = Word2Vec(tokenized_X_test_arr, vector_size = 150, workers = 4, sg = 1)
X_test_wv_skipgram = X_test.apply(lambda text: np.mean([Word2Vec_model_Xtest_skipgram.wv[word] for word in text.split() if word in Word2Vec_model_Xtest_skipgram.wv]))

/usr/local/lib/python3.12/dist-packages/numpy/_core/fromnumeric.py:3596: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.12/dist-packages/numpy/_core/_methods.py:138: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [ ]:
print(type(X_test_wv_skipgram))
print('-'*60)
y_test = pd.Series(y_test)
#type(y_train)
test_skipgram_data = pd.concat([X_test_wv_skipgram.reset_index(drop=True), y_test.reset_index(drop=True)], axis=1)
test_skipgram_data.head()

<class 'pandas.core.series.Series'>
------------------------------------------------------------


,text,0
0,NaN,34
1,-0.00533,35
2,NaN,4
3,-0.00213,34
4,NaN,39


In [ ]:
print('Null Values Post Skip Gram Vectorization in Test data:', test_skipgram_data.isna().sum())

test_skipgram_data = test_skipgram_data.dropna()
print('-'*60)
print('Shape of Vectorized Test Set Post removing Null Values : ',test_skipgram_data.shape)

Null Values Post Skip Gram Vectorization in Test data: text    5449
0          0
dtype: int64
------------------------------------------------------------
Shape of Vectorized Test Set Post removing Null Values :  (1065, 2)


In [ ]:
test_skipgram_data = test_skipgram_data.drop_duplicates()
test_skipgram_data.shape

(550, 2)

In [ ]:
train_skipgram_data.to_csv('train_skipgram_data.csv', index = False, encoding = 'utf-8-sig', chunksize=5000)
test_skipgram_data.to_csv('test_skipgram_data.csv', index = False, encoding = 'utf-8-sig', chunksize=5000)

Word2Vec_model_Xtrained_skipgram.save('/content/Word2Vec_model_Xtrain_skipgram_file.model')
Word2Vec_model_Xtest_skipgram. save('/content/Word2Vec_model_Xtest_skipgram_file.model')

train_skipgram_data = pd.read_csv('/content/train_skipgram_data.csv')
test_skipgram_data = pd.read_csv('/content/test_skipgram_data.csv')

In [ ]:
tokenized_X_test_arr = [text.split() for text in X_test_arr]
Word2Vec_model_Xtest_cbow = Word2Vec(tokenized_X_test_arr, vector_size = 150, workers = 4, sg = 0)
X_test_wv_cbow = X_test.apply(lambda text: np.mean([Word2Vec_model_Xtest_cbow.wv[word] for word in text.split() if word in Word2Vec_model_Xtest_cbow.wv]))

tokenized_X_train_arr = [text.split() for text in X_train_arr]
Word2Vec_model_Xtrain_cbow = Word2Vec(tokenized_X_train_arr, vector_size = 150, workers = 4, sg = 0)
X_train_wv_cbow = X_train.apply(lambda text: np.mean([Word2Vec_model_Xtrain_cbow.wv[word] for word in text.split() if word in Word2Vec_model_Xtrain_cbow.wv]))

/usr/local/lib/python3.12/dist-packages/numpy/_core/fromnumeric.py:3596: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.12/dist-packages/numpy/_core/_methods.py:138: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [ ]:

print(type(X_train_wv_cbow))
print('-'*60)
#y_test = pd.Series(y_test)
#type(y_train)
train_cbow_data = pd.concat([X_train_wv_cbow.reset_index(drop=True), y_train.reset_index(drop=True)], axis=1)
train_cbow_data.head()

<class 'pandas.core.series.Series'>
------------------------------------------------------------


,text,0
0,0.010782,39
1,0.012436,25
2,0.048618,39
3,0.017117,13
4,0.026202,17


In [ ]:
print(type(X_test_wv_cbow))
print('-'*60)
#y_train = pd.Series(y_train)
#type(y_train)
test_cbow_data = pd.concat([X_test_wv_cbow.reset_index(drop=True), y_test.reset_index(drop=True)], axis=1)
test_cbow_data.head()

<class 'pandas.core.series.Series'>
------------------------------------------------------------


,text,0
0,0.005803,34
1,0.011609,35
2,0.006282,4
3,0.006317,34
4,0.010145,39


In [ ]:
print('Null Values Post Continuous Bag Of Words Vectorization in Train data:', train_cbow_data.isna().sum())
train_cbow_data = train_cbow_data.dropna()
print('Shape of Vectorized Train Data post removal of null values', train_cbow_data.shape)

Null Values Post Continuous Bag Of Words Vectorization in Train data: text    22
0        0
dtype: int64
Shape of Vectorized Train Data post removal of null values (58595, 2)


In [ ]:
print('Null Values Post Continuous Bag Of Words Vectorization in Test data:', test_cbow_data.isna().sum())
test_cbow_data = test_cbow_data.dropna()
print('Shape of Vectorized Test Data post removal of null values', test_cbow_data.shape)

Null Values Post Continuous Bag Of Words Vectorization in Test data: text    5
0       0
dtype: int64
Shape of Vectorized Test Data post removal of null values (6509, 2)


In [ ]:
train_cbow_data = train_cbow_data.drop_duplicates()
train_cbow_data.shape

(58293, 2)

In [ ]:
test_cbow_data = test_cbow_data.drop_duplicates()
test_cbow_data.shape

(6499, 2)

In [ ]:
train_cbow_data.to_csv('train_cbow_data_file.csv', index = False, encoding = 'utf-8-sig', chunksize=5000)
test_cbow_data.to_csv('test_cbow_data_file.csv', index = False, encoding = 'utf-8-sig', chunksize=5000)

Word2Vec_model_Xtrain_cbow.save('/content/Word2Vec_model_Xtrain_cbow_file.model')
Word2Vec_model_Xtest_cbow.save('/content/Word2Vec_model_Xtest_cbow_file.model')

train_cbow_data = pd.read_csv('/content/train_cbow_data_file.csv')
test_cbow_data = pd.read_csv('/content/test_cbow_data_file.csv')

In [ ]:

X_test_wv_skipgram = test_skipgram_data.text
y_test = test_skipgram_data.iloc[:,1]
X_test_wv_skipgram.shape, y_test.shape

((550,), (550,))

In [ ]:

X_train_wv_skipgram = train_skipgram_data.text
y_train = train_skipgram_data.iloc[:,1]
X_train_wv_skipgram.shape, y_train.shape


((2961,), (2961,))

In [ ]:
type(X_train_wv_skipgram), type(y_train)

(pandas.core.series.Series, pandas.core.series.Series)

In [ ]:
X_train_wv_skipgram = pd.DataFrame(X_train_wv_skipgram)
y_train = pd.DataFrame(y_train)
X_train_wv_skipgram.shape, y_train.shape

((2961, 1), (2961, 1))

In [ ]:
X_test_wv_skipgram = pd.DataFrame(X_test_wv_skipgram)
y_test = pd.DataFrame(y_test)
X_test_wv_skipgram.shape, y_test.shape

((550, 1), (550, 1))

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

# Fit the model on train
model_base = LogisticRegression()
model_base.fit(X_train_wv_skipgram, y_train)
#predict on test
Y_predict = model_base.predict(X_test_wv_skipgram)
print("Accuracy of Logistic Regression Base Model on Test Data without Hyperparameter Tuning = ", model_base.score(X_test_wv_skipgram, y_test))

print('-'*60)

NNH_base = KNeighborsClassifier()
NNH_base.fit(X_train_wv_skipgram, y_train)
Y_predict = NNH_base.predict(X_test_wv_skipgram)
print("Accuracy of KNN Base Model on Test Data without Hyperparameter Tuning = ", NNH_base.score(X_test_wv_skipgram, y_test))

print('-'*60)

svc_base = SVC()
svc_base.fit(X_train_wv_skipgram, y_train)
Y_predict = svc_base.predict(X_test_wv_skipgram)
print("Accuracy of SVC Base Model on Test Data without Hyperparameter Tuning =", svc_base.score(X_test_wv_skipgram, y_test))


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.12/dist-packages/sklearn/neighbors/_classification.py:239: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy of Logistic Regression Base Model on Test Data without Hyperparameter Tuning =  0.2509090909090909
------------------------------------------------------------
Accuracy of KNN Base Model on Test Data without Hyperparameter Tuning =  0.1890909090909091
------------------------------------------------------------
Accuracy of SVC Base Model on Test Data without Hyperparameter Tuning = 0.24909090909090909


In [ ]:
# Hyperparameter Tuning for KNN Model for Skipgram Vectorized Model

from sklearn.model_selection import GridSearchCV

neighbors = range(1,100)
grid_params = { 'n_neighbors' : neighbors,
               'weights' : ['uniform','distance'],
               'metric' : ['minkowski','euclidean','manhattan']}

gs = GridSearchCV(KNeighborsClassifier(), grid_params, verbose = 1, cv=3, n_jobs = -1)

# fit the Grid Search model for KNN model on our train set
g_res = gs.fit(X_train_wv_skipgram, y_train)

Fitting 3 folds for each of 594 candidates, totalling 1782 fits


/usr/local/lib/python3.12/dist-packages/sklearn/neighbors/_classification.py:239: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


In [ ]:
# get the hyperparameters with the above best score
print('Hyperparameters for the KNN Model :', g_res.best_params_)

# find the best score for our KNN model on Train Data after Hyperparameter tuning
print("Best mean accuracy of KNN model on Train Data with Hyperparameter Tuning done = ", g_res.best_score_)

# Accuracy of Hyperparameter tuned KNN model on the Test Data
print("Accuracy for the Test Data with Hyperparameter Tuned KNN Model =", g_res.score(X_test_wv_skipgram, y_test))

Hyperparameters for the KNN Model : {'metric': 'minkowski', 'n_neighbors': 81, 'weights': 'uniform'}
Best mean accuracy of KNN model on Train Data with Hyperparameter Tuning done =  0.26916582235731173
Accuracy for the Test Data with Hyperparameter Tuned KNN Model = 0.2290909090909091


In [ ]:

y_predict_skipgram = g_res.predict(X_test_wv_skipgram)
performance_metrics(y_test, y_predict_skipgram)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         6
           1       0.00      0.00      0.00         8
           4       0.00      0.00      0.00        24
           6       0.00      0.00      0.00         3
           7       0.00      0.00      0.00         2
           8       0.00      0.00      0.00         6
           9       0.00      0.00      0.00         2
          10       0.00      0.00      0.00        19
          11       0.00      0.00      0.00         3
          12       0.00      0.00      0.00         6
          13       0.00      0.00      0.00        30
          14       0.00      0.00      0.00        17
          15       0.00      0.00      0.00         2
          16       0.00      0.00      0.00         6
          17       0.00      0.00      0.00        10
          18       0.00      0.00      0.00         2
          19       0.00      0.00      0.00        15
          21       0.00    

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
# Hyperparameter Tuning for SVM Model for Skipgram Vectorized dataset

# defining parameter range
param_grid = {'C': [0.1, 1, 10, 100, 1000],
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['linear','rbf']}

grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3)

# fitting the model for grid search
best_grid = grid.fit(X_train_wv_skipgram, y_train)

/usr/local/lib/python3.12/dist-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Fitting 5 folds for each of 50 candidates, totalling 250 fits
[CV 1/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.260 total time=   0.4s


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[CV 2/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.260 total time=   0.7s


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


...

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[CV 4/5] END ..C=1000, gamma=0.0001, kernel=rbf;, score=0.260 total time=   0.3s


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[CV 5/5] END ..C=1000, gamma=0.0001, kernel=rbf;, score=0.258 total time=   0.4s


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
# print best parameters after Hyperparameter tuning
print('Hyperparameters for the SVM Model :', best_grid.best_params_)

# find the best score for our SVM model on Train Data after Hyperparameter tuning
print("Best cross-validated mean accuracy of SVM model with Hyperparameter Tuning done = ", best_grid.best_score_)

# Accuracy of Hyperparameter tuned SVM model on the Test Data
print("Accuracy for the Test Data with Hyperparameter Tuned SVM Model =", best_grid.score(X_test_wv_skipgram, y_test))

Hyperparameters for the SVM Model : {'C': 0.1, 'gamma': 1, 'kernel': 'linear'}
Best cross-validated mean accuracy of SVM model with Hyperparameter Tuning done =  0.2597095620072011
Accuracy for the Test Data with Hyperparameter Tuned SVM Model = 0.2509090909090909


In [ ]:
# print best parameters after Hyperparameter tuning
print('Hyperparameters for the SVM Model :', best_grid.best_params_)

# find the best score for our SVM model on Train Data after Hyperparameter tuning
print("Best cross-validated mean accuracy of SVM model with Hyperparameter Tuning done = ", best_grid.best_score_)

# Accuracy of Hyperparameter tuned SVM model on the Test Data
print("Accuracy for the Test Data with Hyperparameter Tuned SVM Model =", best_grid.score(X_test_wv_skipgram, y_test))

Hyperparameters for the SVM Model : {'C': 0.1, 'gamma': 1, 'kernel': 'linear'}
Best cross-validated mean accuracy of SVM model with Hyperparameter Tuning done =  0.2597095620072011
Accuracy for the Test Data with Hyperparameter Tuned SVM Model = 0.2509090909090909


In [ ]:

y_predict_skipgram = best_grid.predict(X_test_wv_skipgram)
performance_metrics(y_test, y_predict_skipgram)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         6
           1       0.00      0.00      0.00         8
           4       0.00      0.00      0.00        24
           6       0.00      0.00      0.00         3
           7       0.00      0.00      0.00         2
           8       0.00      0.00      0.00         6
           9       0.00      0.00      0.00         2
          10       0.00      0.00      0.00        19
          11       0.00      0.00      0.00         3
          12       0.00      0.00      0.00         6
          13       0.00      0.00      0.00        30
          14       0.00      0.00      0.00        17
          15       0.00      0.00      0.00         2
          16       0.00      0.00      0.00         6
          17       0.00      0.00      0.00        10
          18       0.00      0.00      0.00         2
          19       0.00      0.00      0.00        15
          21       0.00    

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
X_test_wv_cbow = test_cbow_data.text
y_test = test_cbow_data.iloc[:,1]
X_test_wv_cbow.shape, y_test.shape

((6499,), (6499,))

In [ ]:
X_train_wv_cbow = train_cbow_data.text
y_train = train_cbow_data.iloc[:,1]
X_train_wv_cbow.shape, y_train.shape

((58293,), (58293,))

In [ ]:
type(X_train_wv_cbow), type(y_train)

(pandas.core.series.Series, pandas.core.series.Series)

In [ ]:

X_train_wv_cbow = pd.DataFrame(X_train_wv_cbow)
y_train = pd.DataFrame(y_train)
X_train_wv_cbow.shape, y_train.shape

((58293, 1), (58293, 1))

In [ ]:

X_test_wv_cbow = pd.DataFrame(X_test_wv_cbow)
y_test = pd.DataFrame(y_test)
X_test_wv_cbow.shape, y_test.shape

((6499, 1), (6499, 1))

In [ ]:
# Hyperparameter Tuning for KNN Model for cbow Vectorized dataset

from sklearn.model_selection import GridSearchCV

neighbors = range(1,100)
grid_params = { 'n_neighbors' : neighbors,
               'weights' : ['uniform','distance'],
               'metric' : ['minkowski','euclidean','manhattan']}

gs = GridSearchCV(KNeighborsClassifier(), grid_params, verbose = 1, cv=3, n_jobs = -1)

# fit the Grid Search model for KNN model on our train set
g_res = gs.fit(X_train_wv_cbow, y_train)

Fitting 3 folds for each of 594 candidates, totalling 1782 fits


/usr/local/lib/python3.12/dist-packages/sklearn/neighbors/_classification.py:239: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


In [ ]:
# get the hyperparameters with the above best score
print('Hyperparameters for the KNN Model :',g_res.best_params_)

# find the best score for our KNN model on Train Data after Hyperparameter tuning
print("Best mean accuracy of KNN model on Train Data with Hyperparameter Tuning done = ", g_res.best_score_)

# Accuracy of Hyperparameter tuned KNN model on the Test Data
print("Accuracy for the Test Data with Hyperparameter Tuned KNN Model =", g_res.score(X_test_wv_cbow, y_test))

Hyperparameters for the KNN Model : {'metric': 'minkowski', 'n_neighbors': 96, 'weights': 'uniform'}
Best mean accuracy of KNN model on Train Data with Hyperparameter Tuning done =  0.3637829585027362
Accuracy for the Test Data with Hyperparameter Tuned KNN Model = 0.3659024465302354


In [ ]:

y_predict_cbow = g_res.predict(X_test_wv_cbow)
performance_metrics(y_test, y_predict_cbow)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        44
           1       0.00      0.00      0.00        43
           2       0.00      0.00      0.00        13
           3       0.00      0.00      0.00        11
           4       0.00      0.00      0.00       308
           5       0.00      0.00      0.00        11
           6       0.00      0.00      0.00        41
           7       0.00      0.00      0.00        16
           8       0.00      0.00      0.00        40
           9       0.00      0.00      0.00        31
          10       0.00      0.00      0.00       199
          11       0.00      0.00      0.00         8
          12       0.00      0.00      0.00        51
          13       0.00      0.00      0.00       277
          14       0.00      0.00      0.00       107
          15       0.00      0.00      0.00         7
          16       0.00      0.00      0.00        43
          17       0.00    

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [111]:
# Hyperparameter Tuning for SVM Model for cbow Vectorized dataset

# defining parameter range
param_grid = {'C': [0.1, 1, 10, 100, 1000],
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['linear','rbf']}

grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3)

# fitting the model for grid search
best_grid = grid.fit(X_train_wv_cbow, y_train)

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Fitting 5 folds for each of 50 candidates, totalling 250 fits
[CV 1/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.370 total time= 1.6min


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[CV 2/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.370 total time= 1.6min


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


...

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[CV 4/5] END ..C=1000, gamma=0.0001, kernel=rbf;, score=0.370 total time= 2.5min


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[CV 5/5] END ..C=1000, gamma=0.0001, kernel=rbf;, score=0.370 total time= 2.6min


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [112]:
# print best parameter after Hyperparameter tuning
print('Hyperparameters for the SVM Model :', best_grid.best_params_)

# find the best score for our SVM model on Train Data after Hyperparameter tuning
print("Best mean accuracy of SVM model on Train Data with Hyperparameter Tuning done = ", best_grid.best_score_)

# Accuracy of Hyperparameter tuned SVM model on the Test Data
print("Accuracy for the Test Data with Hyperparameter Tuned SVM Model =", best_grid.score(X_test_wv_cbow, y_test))

Hyperparameters for the SVM Model : {'C': 0.1, 'gamma': 1, 'kernel': 'linear'}
Best mean accuracy of SVM model on Train Data with Hyperparameter Tuning done =  0.3704046792840604
Accuracy for the Test Data with Hyperparameter Tuned SVM Model = 0.36805662409601475


In [116]:
y_predict_cbow = best_grid.predict(X_test_wv_cbow)
performance_metrics(y_test, y_predict_cbow)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        44
           1       0.00      0.00      0.00        43
           2       0.00      0.00      0.00        13
           3       0.00      0.00      0.00        11
           4       0.00      0.00      0.00       308
           5       0.00      0.00      0.00        11
           6       0.00      0.00      0.00        41
           7       0.00      0.00      0.00        16
           8       0.00      0.00      0.00        40
           9       0.00      0.00      0.00        31
          10       0.00      0.00      0.00       199
          11       0.00      0.00      0.00         8
          12       0.00      0.00      0.00        51
          13       0.00      0.00      0.00       277
          14       0.00      0.00      0.00       107
          15       0.00      0.00      0.00         7
          16       0.00      0.00      0.00        43
          17       0.00    

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [114]:
param={'alpha': [0.00001, 0.0001, 0.001, 0.1, 1, 10, 100,1000]}

gs_clf = GridSearchCV(MultinomialNB(),param,cv=10,return_train_score=True)
best_clf = gs_clf.fit(X_train_copy, y_train_copy)

In [115]:
# print best parameter after Hyperparameter tuning
print('Hyperparameters for the Multinomial NB Model :',best_clf.best_params_)

# find the best score for our Multinomial NB model on Train Data after Hyperparameter tuning
print("Best mean accuracy of Multinomial NB model on Train Data with Hyperparameter Tuning done = ", best_clf.best_score_)

# Accuracy of Hyperparameter tuned Multinomial NB model on the Test Data
print("Accuracy for the Test Data with Hyperparameter Tuned Multinomial NB Model =", best_clf.score(X_test_copy, y_test_copy))

Hyperparameters for the Multinomial NB Model : {'alpha': 0.1}
Best mean accuracy of Multinomial NB model on Train Data with Hyperparameter Tuning done =  0.3985021414154397
Accuracy for the Test Data with Hyperparameter Tuned Multinomial NB Model = 0.39990789069696037


In [117]:
y_predict_copy = best_clf.predict(X_test_copy)
performance_metrics(y_test_copy, y_predict_copy)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        44
           1       0.00      0.00      0.00        43
           2       0.00      0.00      0.00        13
           3       0.00      0.00      0.00        11
           4       0.40      0.01      0.01       308
           5       0.00      0.00      0.00        11
           6       0.00      0.00      0.00        41
           7       0.00      0.00      0.00        16
           8       0.00      0.00      0.00        40
           9       0.00      0.00      0.00        31
          10       0.00      0.00      0.00       199
          11       0.00      0.00      0.00         8
          12       0.00      0.00      0.00        51
          13       0.00      0.00      0.00       277
          14       0.00      0.00      0.00       107
          15       0.00      0.00      0.00         7
          16       0.00      0.00      0.00        43
          17       0.00    

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


5.B. Multinomial NB Supervised ML algorithm gave the highest Train & Test Accuracy of 40.37 & 41 respectively.

Probable Reason :

The Multinomial Naive Bayes can be accepted as the probabilistic approach to classifying documents in the case of acknowledging the frequency of a specified word in a text document.
simple and robust.
assigns documents to classes based on the statistical analysis of their contents.
relies on an assumption of conditional independence of features given a class, which is often a good first approximation to real-world phenomena.
they predict the tag of a text. They calculate the probability of each tag for a given text and then output the tag with the highest one.
5.A. TFID performed better since it worked with Multinomial NB algorithm. Negative values in Word2Vec - Skipgram & CBOW vectorization were not compatible with Multinomial NB ML algorithm.

5.C. Multinomial NB Hyperparameter alpha = 0.001 helped to improve the performance of the algorithm. Probable Reason : Algorithm's ability in classifying unseen words during training - for more complex sentences and by that I mean for sentences that may be more and more out of context relative to the training set, the noise will increase. In such cases increasing 'alpha' will be needed to smooth out the class separation.

5.D. F1-Score to be given highest importance for multi-class classification since it gives a trade-off between Precision & Recall. Also the classes here have high data imbalance

Part 2

In [118]:

import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
import seaborn as sns

import logging

import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize

import gc

In [119]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [121]:
with open('/content/drive/My Drive/Projects//NLP P1/GL Bot.json') as file:
  data = json.load(file)

In [122]:
print(data)

{'intents': [{'tag': 'Intro', 'patterns': ['hi', 'how are you', 'is anyone there', 'hello', 'whats up', 'hey', 'yo', 'listen', 'please help me', 'i am learner from', 'i belong to', 'aiml batch', 'aifl batch', 'i am from', 'my pm is', 'blended', 'online', 'i am from', 'hey ya', 'talking to you for first time'], 'responses': ['Hello! how can i help you ?'], 'context_set': ''}, {'tag': 'Exit', 'patterns': ['thank you', 'thanks', 'cya', 'see you', 'later', 'see you later', 'goodbye', 'i am leaving', 'have a Good day', 'you helped me', 'thanks a lot', 'thanks a ton', 'you are the best', 'great help', 'too good', 'you are a good learning buddy'], 'responses': ['I hope I was able to assist you, Good Bye'], 'context_set': ''}, {'tag': 'Olympus', 'patterns': ['olympus', 'explain me how olympus works', 'I am not able to understand olympus', 'olympus window not working', 'no access to olympus', 'unable to see link in olympus', 'no link visible on olympus', 'whom to contact for olympus', 'lot of p

In [120]:
#import nltk
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

import numpy
#import tflearn
import tensorflow
import random
import json

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [123]:
print (data['intents'])

[{'tag': 'Intro', 'patterns': ['hi', 'how are you', 'is anyone there', 'hello', 'whats up', 'hey', 'yo', 'listen', 'please help me', 'i am learner from', 'i belong to', 'aiml batch', 'aifl batch', 'i am from', 'my pm is', 'blended', 'online', 'i am from', 'hey ya', 'talking to you for first time'], 'responses': ['Hello! how can i help you ?'], 'context_set': ''}, {'tag': 'Exit', 'patterns': ['thank you', 'thanks', 'cya', 'see you', 'later', 'see you later', 'goodbye', 'i am leaving', 'have a Good day', 'you helped me', 'thanks a lot', 'thanks a ton', 'you are the best', 'great help', 'too good', 'you are a good learning buddy'], 'responses': ['I hope I was able to assist you, Good Bye'], 'context_set': ''}, {'tag': 'Olympus', 'patterns': ['olympus', 'explain me how olympus works', 'I am not able to understand olympus', 'olympus window not working', 'no access to olympus', 'unable to see link in olympus', 'no link visible on olympus', 'whom to contact for olympus', 'lot of problem with 

In [124]:
for each in data['intents']:
  print(each['tag'])
  print(each['patterns'])
  print(each['responses'])

Intro
['hi', 'how are you', 'is anyone there', 'hello', 'whats up', 'hey', 'yo', 'listen', 'please help me', 'i am learner from', 'i belong to', 'aiml batch', 'aifl batch', 'i am from', 'my pm is', 'blended', 'online', 'i am from', 'hey ya', 'talking to you for first time']
['Hello! how can i help you ?']
Exit
['thank you', 'thanks', 'cya', 'see you', 'later', 'see you later', 'goodbye', 'i am leaving', 'have a Good day', 'you helped me', 'thanks a lot', 'thanks a ton', 'you are the best', 'great help', 'too good', 'you are a good learning buddy']
['I hope I was able to assist you, Good Bye']
Olympus
['olympus', 'explain me how olympus works', 'I am not able to understand olympus', 'olympus window not working', 'no access to olympus', 'unable to see link in olympus', 'no link visible on olympus', 'whom to contact for olympus', 'lot of problem with olympus', 'olypus is not a good tool', 'lot of problems with olympus', 'how to use olympus', 'teach me olympus']
['Link: Olympus wiki']
SL
[

In [125]:
for each in data['intents']:
  print(each['tag'])

Intro
Exit
Olympus
SL
NN
Bot
Profane
Ticket


In [126]:
new_line = {'tag':'Course',
          'patterns':['what are the courses available','topics covered', 'better course', 'course duration'],
          'responses':['Link:List of Course'],
          'context_set': ''}

new_line_1 = {'tag':'Career',
          'patterns':['what are the career opportunities','roles offered','campus placement','salary offered'],
          'responses':['Link:List of sample Roles offered'],
          'context_set': ''}

In [127]:
data['intents'].append(new_line)
data['intents'].append(new_line_1)

for each in data['intents']:
  print(each['tag'])

Intro
Exit
Olympus
SL
NN
Bot
Profane
Ticket
Course
Career


In [128]:
for each in data['intents']:
  print(each['tag'])
  print(each['patterns'])
  print(each['responses'])

Intro
['hi', 'how are you', 'is anyone there', 'hello', 'whats up', 'hey', 'yo', 'listen', 'please help me', 'i am learner from', 'i belong to', 'aiml batch', 'aifl batch', 'i am from', 'my pm is', 'blended', 'online', 'i am from', 'hey ya', 'talking to you for first time']
['Hello! how can i help you ?']
Exit
['thank you', 'thanks', 'cya', 'see you', 'later', 'see you later', 'goodbye', 'i am leaving', 'have a Good day', 'you helped me', 'thanks a lot', 'thanks a ton', 'you are the best', 'great help', 'too good', 'you are a good learning buddy']
['I hope I was able to assist you, Good Bye']
Olympus
['olympus', 'explain me how olympus works', 'I am not able to understand olympus', 'olympus window not working', 'no access to olympus', 'unable to see link in olympus', 'no link visible on olympus', 'whom to contact for olympus', 'lot of problem with olympus', 'olypus is not a good tool', 'lot of problems with olympus', 'how to use olympus', 'teach me olympus']
['Link: Olympus wiki']
SL
[

In [131]:
import nltk
nltk.download('punkt_tab')
words = []
labels = []
docs_x = []
docs_y = []

for intent in data['intents']:
  for pattern in intent['patterns']:
    wrds = nltk.word_tokenize(pattern)
    words.extend(wrds)
    docs_x.append(wrds)
    docs_y.append(intent["tag"])

  if intent['tag'] not in labels:
    labels.append(intent['tag'])

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [132]:
labels

['Intro',
 'Exit',
 'Olympus',
 'SL',
 'NN',
 'Bot',
 'Profane',
 'Ticket',
 'Course',
 'Career']

In [133]:
words = [stemmer.stem(w.lower()) for w in words if w != "?"]
words = sorted(list(set(words)))

labels = sorted(labels)

In [134]:
training = []
output = []

out_empty = [0 for _ in range(len(labels))]

for x, doc in enumerate(docs_x):
  bag = []
  wrds = [stemmer.stem(w.lower()) for w in doc]

  for w in words:
    if w in wrds:
      bag.append(1)
    else:
      bag.append(0)

  output_row = out_empty[:]
  output_row[labels.index(docs_y[x])] = 1

  training.append(bag)
  output.append(output_row)

In [135]:

len(docs_x)

136

In [136]:
len(words)

163

In [137]:
training = numpy.array(training)
output = numpy.array(output)

In [139]:
import random

import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense,Input,BatchNormalization,Activation,Dropout
from tensorflow.keras.utils import to_categorical
from keras import optimizers,regularizers, losses
from tensorflow.keras import backend

backend.clear_session()

random.seed(1)
np.random.seed(1)
tensorflow.random.set_seed(1)

In [140]:
training

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [141]:
output

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 1, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0]])

In [142]:
training.shape, output.shape

((136, 163), (136, 10))

In [143]:
model=Sequential()
model.add(Dense(100,input_dim=len(training[0]),activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10,activation='softmax'))

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [144]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 100)            │        16,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         6,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,514 (91.85 KB)

 Trainable params: 23,514 (91.85 KB)

 Non-trainable params: 0 (0.00 B)

In [146]:
sgd = optimizers.SGD(learning_rate=0.001, decay=1e-6, momentum=0.9)
adam = tensorflow.keras.optimizers.Adam(learning_rate=0.1)
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=["accuracy"])

/usr/local/lib/python3.12/dist-packages/keras/src/optimizers/base_optimizer.py:86: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


In [147]:
history = model.fit(training, output, epochs = 256) #batch_size = 32

Epoch 1/256
5/5 ━━━━━━━━━━━━━━━━━━━━ 4s 15ms/step - accuracy: 0.1960 - loss: 2.4243
Epoch 2/256
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.2263 - loss: 2.5061
Epoch 3/256
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.3834 - loss: 2.1360
Epoch 4/256
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.4478 - loss: 1.8221 
Epoch 5/256
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.4484 - loss: 2.3476 
Epoch 6/256
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.4218 - loss: 2.0147
Epoch 7/256
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.5116 - loss: 2.0346
Epoch 8/256
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.5067 - loss: 1.9071
Epoch 9/256
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.4799 - loss: 1.9139
Epoch 10/256
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.5442 - loss: 1.8495
Epoch 11/256
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.5551 - loss: 2.0361
Epoch 12/256
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.5581 - lo

In [148]:
def bag_of_words(s, words):
  bag = [0 for _ in range(len(words))]
  #bag=[]

  s_words = nltk.word_tokenize(s)
  s_words = [stemmer.stem(word.lower()) for word in s_words]

  for se in s_words:
    for i, w in enumerate(words):
      if w == se:
        bag[i] = 1

  return np.array(bag)

In [149]:
def chat():
  print("Hi! I am a chatbot created by Great Learning for your service. (type quit to stop)! \n If answer is not right, type : *")
  while True:
    inp = input("You: ")
    if inp.lower()=="*":
      print("Please rephrase your question an type again.")
    if inp.lower() == "quit":
      break
    pp = bag_of_words(inp,words)
    train = pp.reshape(163,1)
    train = tensorflow.expand_dims(train,axis=0)
    print(train.shape)
    results = model.predict(train)

    #results = model.predict([bag_of_words(inp, words)])
    results_index = np.argmax(results)
    tag = labels[results_index]

    for tg in data["intents"]:
      if tg['tag'] == tag:
        responses = tg['responses']

    print(random.choice(responses))

In [150]:
chat()

Hi! I am a chatbot created by Great Learning for your service. (type quit to stop)! 
 If answer is not right, type : *
You: Hey There
(1, 163, 1)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
Hello! how can i help you ?
You: I have a doubt to be clarified in SVG Regression
(1, 163, 1)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
Hello! how can i help you ?
You: I have a doubt to be clarified in Logistic Regression
(1, 163, 1)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
Link: Machine Learning wiki 
You: List of sample Roles offered
(1, 163, 1)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
Hello! how can i help you ?
You: Olympus 
(1, 163, 1)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
Link: Olympus wiki
You: Deep Learning
(1, 163, 1)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
Link: Neural Nets wiki
You: Quit
